In [1]:
import cvxpy as cp
import numpy as np

In [62]:
# Initialize price parameters
prices = [0.]*3*12+[1.]*3*12
T = len(prices)

# Initialize variables
SOC = cp.Variable((T+1,))
x = cp.Variable((T,))

# Initialize constraints
constraints = [SOC[0]==2.]
constraints += [SOC[T]==2.]
for t in range(T):
    constraints += [-1.<=x[t], x[t]<=1., 1.<=SOC[t], SOC[t]<=5., SOC[t+1]==SOC[t]+(x[t]/12)]

# Objective function
obj = cp.sum(cp.multiply(prices,-x))
obj = cp.Maximize(obj)

In [63]:
# Solve problem
prob = cp.Problem(obj, constraints)
prob.solve() #returns optimal value = prob.value

35.99999999657463

In [64]:
# Print output and solution characteristics
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal soc", SOC.value)
print("optimal var", x.value)

status: optimal
optimal value 35.99999999657463
optimal soc [2.         2.08333333 2.16666667 2.25       2.33333333 2.41666667
 2.5        2.58333333 2.66666667 2.75       2.83333333 2.91666667
 3.         3.08333333 3.16666667 3.25       3.33333333 3.41666667
 3.5        3.58333333 3.66666667 3.75       3.83333333 3.91666667
 4.         4.08333333 4.16666667 4.25       4.33333333 4.41666667
 4.5        4.58333333 4.66666667 4.75       4.83333333 4.91666667
 5.         4.91666667 4.83333333 4.75       4.66666667 4.58333333
 4.5        4.41666667 4.33333333 4.25       4.16666667 4.08333333
 4.         3.91666667 3.83333333 3.75       3.66666667 3.58333333
 3.5        3.41666667 3.33333333 3.25       3.16666667 3.08333333
 3.         2.91666667 2.83333333 2.75       2.66666667 2.58333333
 2.5        2.41666667 2.33333333 2.25       2.16666667 2.08333333
 2.        ]
optimal var [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  